In [1]:
!pip install torch
!pip install casadi

     |████████████████████████████████| 34.2MB 114kB/s 


In [1]:
import casadi as ca
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
a = np.zeros(3,10)
a

TypeError: Cannot interpret '10' as a data type

In [8]:
def mix2motor(u_mix):
    '''from input of control to each motor duty'''
    map = np.linalg.inv(np.array([
        [-1,1,1,-1],
        [1,-1,1,-1],
        [1,1,-1,-1],
        [1,1,1,1]]))
    return ca.mtimes(map,u_mix)

def saturate(motor: ca.SX, len: int) -> ca.SX:
    ''' saturate the input motor voltage '''
    for i in range(4):
        v = motor[i]
        v = ca.if_else(v > 1, 1, ca.if_else(v < 0, 0, v))
        motor[i] = v
    return motor


In [9]:
u_mix = ca.SX.sym('u_mix', 4)
u_motor = saturate(mix2motor(u_mix), 4)                           # convert u_mix(angle input) to motor duty

u_motor

SX(@1=1, @2=-0.25, @3=0.25, @4=((((@2*u_mix_0)+(@3*u_mix_1))+(@3*u_mix_2))+(@3*u_mix_3)), @5=(@1<@4), @6=0, @7=((((@3*u_mix_0)+(@2*u_mix_1))+(@3*u_mix_2))+(@3*u_mix_3)), @8=(@1<@7), @9=((((@3*u_mix_0)+(@3*u_mix_1))+(@2*u_mix_2))+(@3*u_mix_3)), @10=(@1<@9), @11=((((@2*u_mix_0)+(@2*u_mix_1))+(@2*u_mix_2))+(@3*u_mix_3)), @12=(@1<@11), [((@5?@1:0)+((!@5)?((!(@4<@6))?@4:0):0)), ((@8?@1:0)+((!@8)?((!(@7<@6))?@7:0):0)), ((@10?@1:0)+((!@10)?((!(@9<@6))?@9:0):0)), ((@12?@1:0)+((!@12)?((!(@11<@6))?@11:0):0))])

In [31]:
mix2motor(np.array([[0],[0],[0],[12]]))

DM([3, 3, 3, 3])

In [16]:
def thrust(throttle, rho, r, V, kV, CT, Cm):
    """
    throttle: 0-1 controls voltage, []
    rho: density of air, [kg/m^3]
    r: radius of propeller, [m]
    V: voltage of battery [V]
    kV: motor kV constant, [rpm/V]
    """
    omega = throttle*V*kV*(2*ca.pi/60)
    q = 0.5*rho*(omega*r)**2
    s = ca.pi*r**2

    return CT*q*s,Cm*q*s

In [36]:
thrust(0, 1.224, 0.1, 11.1, 1550, 1.0e-2, 1e-4)

(0.0, 0.0)

In [33]:
np.array([[1,0,0],[0,0,1],[1,1,1]]).T

array([[1, 0, 1],
       [0, 0, 1],
       [0, 1, 1]])

In [3]:
map = np.linalg.inv(np.array([
        [-1,1,1,-1],
        [1,-1,1,-1],
        [1,1,-1,-1],
        [1,1,1,1]]))
map

array([[-0.25,  0.25,  0.25,  0.25],
       [ 0.25, -0.25,  0.25,  0.25],
       [ 0.25,  0.25, -0.25,  0.25],
       [-0.25, -0.25, -0.25,  0.25]])

In [10]:
1.0 == True

True

In [48]:
a = [1,0,1,1,1,0,0,0,1]
p = torch.FloatTensor([[1.0] if a_b else [0.0] for a_b in a])

pp = torch.FloatTensor()
for i in a:
    if i:
        pn = torch.FloatTensor([[1.0]])
    else:
        pn = torch.FloatTensor([[0.0]])
    pp = torch.cat((pp,pn), dim=0)
1-pp

tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.]])

In [22]:
a = np.array([])
a

array([], dtype=float64)

In [2]:
a = np.random.randint(0,10,(5,12))

In [45]:
a[4].sum()

43

In [46]:
a[2] = np.array([[]])

ValueError: could not broadcast input array from shape (0) into shape (12)

In [3]:
a

array([[3, 7, 0, 4, 0, 6, 6, 9, 3, 3, 1, 5],
       [8, 1, 9, 5, 6, 1, 3, 3, 4, 4, 0, 5],
       [6, 2, 6, 8, 6, 5, 7, 8, 0, 1, 1, 4],
       [5, 4, 7, 8, 0, 1, 1, 7, 2, 1, 8, 9],
       [7, 3, 3, 5, 6, 2, 9, 4, 5, 2, 6, 3]])

In [13]:
a[0,:4].mean()

3.5